In [1]:
import pandas as pd
# from pandas import HDFStore,DataFrame
import numpy as np
import matplotlib.pyplot as plt
import pymongo
from pprint import pprint
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

In [2]:
ratings_matrix = pd.read_pickle("Data/test_matrix.pkl")

ratings_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175993,175995,175999,176001,176003,176007,176075,176157,176211,176271
userId,,,,,,,,,,,,,,,,,,,,,
9,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# preds.to_hdf('Data/preds_hdf2.h5', key='preds',complib='blosc',complevel=9, mode='w')

ratings_matrix.to_hdf('Data/ratings_mx.h5', key='ratings_matrix',complib='blosc',complevel=9, mode='w' )

In [12]:
#load movies

movies = pd.read_csv('Data/movies_sml.csv')

In [13]:
#load ratings

ratings = pd.read_csv('Data/ratings_sml.csv')

In [14]:
R = ratings_matrix.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [6]:
# #load other vectors needed to compute predictions

# U = joblib.load('Data/U.pkl')
# Vt = joblib.load('Data/Vt.pkl')
# sigma = joblib.load('Data/sigma.pkl')

In [7]:
# # calculate the dot product of our vectors
# all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [16]:
all_user_predicted_ratings = joblib.load('Data/all_user_predicted_ratings')

In [17]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = ratings_matrix.columns)
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175993,175995,175999,176001,176003,176007,176075,176157,176211,176271
0,4.043145,-0.031401,-0.109841,-0.040066,-0.007383,-0.362918,0.138744,-0.059072,0.099973,0.204405,...,-0.002675,-0.002703,-0.002703,-0.002703,-0.002732,-0.002617,-0.002744,-0.003168,-0.003729,-0.003049
1,2.967341,0.112618,-0.650235,0.084441,-0.542322,0.190666,-0.334271,-0.013707,-0.149256,0.302472,...,0.001898,0.001875,0.001875,0.001875,0.001851,0.001945,0.002245,0.001783,0.003136,0.002110
2,0.355257,-0.414077,0.207500,-0.050163,0.178221,2.574964,0.237428,-0.055524,0.136858,0.469555,...,-0.000418,-0.000219,-0.000219,-0.000219,-0.000020,-0.000816,-0.000617,-0.000151,-0.001692,-0.000792
3,-0.215976,-0.141312,-0.218790,0.037475,-0.053217,0.369246,-0.140024,-0.031138,-0.099683,-0.607257,...,0.006418,0.006385,0.006385,0.006385,0.006352,0.006485,0.006283,0.007180,0.004884,0.005402
4,1.542347,0.393789,-0.016029,0.048731,-0.132489,0.158159,-0.108102,0.055406,-0.064908,0.346607,...,0.000917,0.001000,0.001000,0.001000,0.001083,0.000750,0.001514,0.000832,0.001296,0.000532


In [47]:
# preds.to_hdf('Data/preds_hdf.h5', key='preds', mode='w')

In [46]:
preds.to_hdf('Data/preds_hdf2.h5', key='preds',complib='blosc',complevel=9, mode='w')

In [18]:
def recommend_movies(predictions, userId, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userId - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userId)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

#     print (f"User {0} has already rated {1} movies. format(userID, user_full.shape[0]")
#     print 'Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations)
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    rec_movies = recommendations['Title']
    already_rated = user_full['Title_x']
    return  already_rated, rec_movies

In [19]:
already_rated, predictions = recommend_movies(preds, 60, movies, ratings, 20)

In [20]:
predictions

423                       Schindler's List
215     Star Wars: Episode IV - A New Hope
293                           Forrest Gump
102                             Braveheart
6587                             Inception
6003                                    Up
5575                                WALL·E
1420                     Life Is Beautiful
1950                             Gladiator
8495                          Interstellar
249                 Léon: The Professional
485                                  Fargo
3110                          Finding Nemo
7820                      Django Unchained
8928                    The Imitation Game
5957                  Inglourious Basterds
5748                   Slumdog Millionaire
5059                           Ratatouille
2950                   Catch Me If You Can
9237                            Inside Out
Name: Title, dtype: object

## Content Filtering Section

In [21]:
# read in pre_processed file movies_bow

movies_bow = pd.read_csv('Data/movies_bow.csv')
movies_bow.head()

,Title,movieId,bag_of_words
0,Toy Story,1,tomhanks timallen donrickles jimvarney johnlas...
1,Jumanji,2,robinwilliams jonathanhyde kirstendunst bradle...
2,Grumpier Old Men,3,waltermatthau jacklemmon sophialoren ann-margr...
3,Waiting to Exhale,4,whitneyhouston angelabassett lorettadevine lel...
4,Father of the Bride Part II,5,stevemartin dianekeaton martinshort kimberlywi...


In [22]:
# Initializing and generating the count matrix 

count = CountVectorizer()
count_matrix = count.fit_transform(movies_bow['bag_of_words'])

In [23]:
# Generating the cosine similarity matrix

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [24]:
#map index to Title. This will be used in the function and is integral to 
# how we access the recommended movies

indices = pd.Series(movies_bow.index, index = movies_bow['Title'])

#Create recommendation function

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:26]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_bow['Title'].iloc[movie_indices]

In [25]:
#test
get_recommendations('Rumble in the Bronx')

11663                   Kung Fu Yoga
4129                       Who Am I?
3463                     Crime Story
670                       Supercop 2
690      Around the World in 80 Days
691      Around the World in 80 Days
1143                       Mr. Magoo
1416                       Rush Hour
1595                      Black Mask
2008                   Shanghai Noon
2440                     Rush Hour 2
4071                     City Hunter
4976                     Thunderbolt
5320                     Rush Hour 3
9890                    Dragon Blade
11352                      Skiptrace
11463                Railroad Tigers
1226                    Mr. Nice Guy
3052                Shanghai Knights
4397                        Gorgeous
4649                New Police Story
6441               Shinjuku Incident
6894              Little Big Soldier
4974                    Swordsman II
0                          Toy Story
Name: Title, dtype: object

## Create Hybrid


In [26]:
# create a copy of the movies dataframe to use in function

movies_map = movies.copy()
movies_map.reset_index(inplace=True)
movies_map.head()

,index,movieId,Title,genres,Actors,Director,Plot,Poster
0,0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",John Lasseter,A cowboy doll is profoundly threatened and jea...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,1,2,Jumanji,Adventure|Children|Fantasy,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,When two kids find and play a magical board ga...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,2,3,Grumpier Old Men,Comedy|Romance,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",Howard Deutch,John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,3,4,Waiting to Exhale,Comedy|Drama|Romance,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"Based on Terry McMillan's novel, this film fol...",https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,4,5,Father of the Bride Part II,Comedy,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,George Banks must deal not only with the pregn...,https://m.media-amazon.com/images/M/MV5BOTEyNz...


In [27]:
movies_map.shape

(12008, 8)

In [ ]:
# def recommend_movies(predictions, userId, movies, original_ratings, num_recommendations):
    
#     # Get and sort the user's predictions
#     user_row_number = userId - 1 # User ID starts at 1, not 0
#     sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
#     # Get the user's data and merge in the movie information.
#     user_data = original_ratings[original_ratings.userId == (userId)]
#     user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
#                      sort_values(['rating'], ascending=False)
#                  )

# #     print (f"User {0} has already rated {1} movies. format(userID, user_full.shape[0]")
# #     print 'Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations)
    
#     # Recommend the highest predicted rating movies that the user hasn't seen yet.
#     recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
#          merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
#                left_on = 'movieId',
#                right_on = 'movieId').
#          rename(columns = {user_row_number: 'Predictions'}).
#          sort_values('Predictions', ascending = False).
#                        iloc[:num_recommendations, :-1]
#                       )
#     rec_movies = recommendations['Title']
#     already_rated = user_full['Title_x']
#     return  already_rated, rec_movies

In [ ]:
 indices = pd.Series(movies_bow.index, index = movies_bow['Title'])

In [36]:
def hybrid(userId, title, num_recommendations, original_ratings, predictions):
    
    #utilize content model to find the top 25 similar movies
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    movies_df = movies_map.iloc[movie_indices][['Title', 'id']]
    
    #get user profile
    user_row_number = userId - 1 # User ID starts at 1, not 0
    
    #get all predictions based on userprofile
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts
    
    #find the movies that the user already saw so we won't recommend it to them again
    user_data = original_ratings[original_ratings.userId == (userId)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False))
    
    #find the movies that the user has not seen
    movie_predicts = (movies[~movies['movieId'].isin(user_full['movieId'])].
    merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}))
                       
     #combine movie predicts and content rating results
    recs_join = pd.merge(movies_df, movie_predicts, how = 'left', 
                         left_on = 'Title', right_on = 'Title').rename(columns = {'Title_x': 'Title'}).                   
                        sort_values('Predictions', ascending = False)
                       
     
     top10 = recs_join['Title'][:11]
                       
#     movies['est'] = movies_df['id'].apply(lambda x: svd.predict(userId, movies_map.loc[x]['movieId']).est)
#     movies = movies.sort_values('est', ascending=False)
#    
     return top10

SyntaxError: invalid syntax (<ipython-input-36-f2003dba8800>, line 31)